In [ ]:
import cv2
import numpy as np
from PIL import Image
from IPython.display import display, clear_output
from adafruit_servokit import ServoKit
import threading
import time
import ipywidgets as widgets
from IPython.display import display

# Initialize ServoKit for controlling servos on 16 channels
kit = ServoKit(channels=16)
# Set initial pan and tilt angles
pan = 20
tilt = 20

# Set initial position for servo motors
# Channel 0 controls Pan, Channel 1 controls Tilt
kit.servo[0].angle = pan
kit.servo[1].angle = tilt


# Function to convert BGR image to JPEG format
def bgr8_to_jpeg(value, quality=75):
    return Image.fromarray(cv2.cvtColor(value, cv2.COLOR_BGR2RGB))


# Initialize Picamera (dummy initialization here; please replace with actual initialization for your environment)
class Picamera2:
    def create_preview_configuration(self, *args, **kwargs):
        return {}

    def configure(self, config):
        pass

    def start(self):
        pass

    def capture_array(self):
        # Dummy image generation for example purposes
        return np.zeros((240, 320, 3), dtype=np.uint8)

    def stop(self):
        pass


output = widgets.Output()
display(output)


# Function for video display and object tracking
def video_display(stop_event):
    global pan
    global tilt
    width, height = 320, 240  # Example resolution

    try:
        picamera = Picamera2()
        config = picamera.create_preview_configuration(
            main={"format": 'RGB888', "size": (320, 240)},
            raw={"format": "SRGGB12", "size": (1920, 1080)}
        )
        picamera.configure(config)
        picamera.start()

        while not stop_event.is_set():
            frame = picamera.capture_array()
            hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

            # Dummy slider values
            hueLow, hueUp = 96, 120
            hue2Low, hue2Up = 50, 0
            Ls, Us = 157, 255
            Lv, Uv = 100, 255

            I_b = np.array([hueLow, Ls, Lv])
            u_b = np.array([hueUp, Us, Uv])
            I_b2 = np.array([hue2Low, Ls, Lv])
            u_b2 = np.array([hue2Up, Us, Uv])
            FGmask = cv2.inRange(hsv, I_b, u_b)
            FGmask2 = cv2.inRange(hsv, I_b2, u_b2)
            FGmaskComp = cv2.add(FGmask, FGmask2)

            contours, _ = cv2.findContours(FGmaskComp, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            contours = sorted(contours, key=lambda x: cv2.contourArea(x), reverse=True)

            # Object tracking and servo adjustment
            for cnt in contours:
                area = cv2.contourArea(cnt)
                (x, y, w, h) = cv2.boundingRect(cnt)
                if area >= 50:
                    cv2.rectangle(frame, (x, y), (x + w, y + h), (255, 0, 0), 3)
                    objX = x + w / 2
                    objY = y + h / 2
                    errorPan = objX - width / 2
                    errorTilt = objY - height / 2
                    if abs(errorPan) > 15:
                        pan = pan - errorPan / 75
                    if abs(errorTilt) > 15:
                        tilt = tilt - errorTilt / 75
                    kit.servo[0].angle = 180 - pan
                    kit.servo[1].angle = 180 - tilt
                break

            # Display frame
            frame_img = bgr8_to_jpeg(frame)
            with output:
                clear_output(wait=True)
                display(frame_img)

            time.sleep(0.03)
    except Exception as e:
        print(f"ビデオ表示中のエラー: {e}")
    finally:
        picamera.stop()


# Function to manually stop the video display
def stop_video_display():
    stop_event.set()
    video_thread.join()


# Start the video display in a separate thread
stop_event = threading.Event()
video_thread = threading.Thread(target=video_display, args=(stop_event,))
video_thread.start()

# Button to stop video display
stop_button = widgets.Button(description="Stop Video Display")
stop_button.on_click(lambda x: stop_video_display())
display(stop_button)
